<img src="http://oproject.org/tiki-download_file.php?fileId=8&display&x=450&y=128">
<img src="http://gfif.udea.edu.co/root/tmva/img/tmva_logo.gif">

# DataLoader Example

## Declare Factory

In [ ]:
TMVA::Tools::Instance();

TFile *inputFile = TFile::Open( "inputdata.root"); 
TFile* outputFile = TFile::Open( "TMVAOutput.root", "RECREATE" );

TMVA::Factory *factory = new TMVA::Factory("TMVAClassification", outputFile,
                             "!V:ROC:!Correlations:!Silent:Color:!DrawProgressBar:AnalysisType=Classification" );

## Declare DataLoader(s)

In [ ]:
TMVA::DataLoader *loader1=new TMVA::DataLoader("dataset1");

loader1->AddVariable( "var1", 'F' );
loader1->AddVariable( "var2", 'F' );
loader1->AddVariable( "var3", 'F' );

TMVA::DataLoader *loader2=new TMVA::DataLoader("dataset2"); 

loader2->AddVariable( "var1", 'F' );
loader2->AddVariable( "var2", 'F' );
loader2->AddVariable( "var3", 'F' );
loader2->AddVariable( "var4", 'F' );
loader2->AddVariable( "var5 := var1-var3", 'F' );
loader2->AddVariable( "var6 := var1+var2", 'F' );

## Setup Dataset(s)

In [ ]:
TTree *tsignal     = (TTree*)inputFile->Get("Sig");
TTree *tbackground = (TTree*)inputFile->Get("Bkg");   

TCut mycuts = "";
TCut mycutb = "";
   
loader1->AddSignalTree    ( tsignal,     1.0 );   //signal weight  = 1
loader1->AddBackgroundTree( tbackground, 1.0 );   //background weight = 1 
loader1->PrepareTrainingAndTestTree( mycuts, mycutb,
"nTrain_Signal=1000:nTrain_Background=1000:SplitMode=Random:NormMode=NumEvents:!V" ); 

loader2->AddSignalTree    ( tsignal,     1.0 );   //signal weight  = 1
loader2->AddBackgroundTree( tbackground, 1.0 );   //background weight = 1   
loader2->PrepareTrainingAndTestTree( mycuts, mycutb,
"nTrain_Signal=1000:nTrain_Background=1000:SplitMode=Random:NormMode=NumEvents:!V" );   


# Booking Methods

## First dataset

In [ ]:
//Boosted Decision Trees
factory->BookMethod(loader1,TMVA::Types::kBDT, "BDT",
"!V:NTrees=200:MinNodeSize=2.5%:MaxDepth=2:BoostType=AdaBoost:AdaBoostBeta=0.5:UseBaggedBoost:BaggedSampleFraction=0.5:SeparationType=GiniIndex:nCuts=20" );

//Multi-Layer Perceptron (Neural Network)
factory->BookMethod(loader1, TMVA::Types::kMLP, "MLP", "!H:!V:NeuronType=tanh:VarTransform=N:NCycles=100:HiddenLayers=N+5:TestRate=5:!UseRegulator" );

## Second dataset

In [ ]:
//Boosted Decision Trees
factory->BookMethod(loader2,TMVA::Types::kBDT, "BDT",
"!V:NTrees=200:MinNodeSize=2.5%:MaxDepth=2:BoostType=AdaBoost:AdaBoostBeta=0.5:UseBaggedBoost:BaggedSampleFraction=0.5:SeparationType=GiniIndex:nCuts=20" );

//Multi-Layer Perceptron
factory->BookMethod(loader2, TMVA::Types::kMLP, "MLP", "!H:!V:NeuronType=tanh:VarTransform=N:NCycles=100:HiddenLayers=N+5:TestRate=5:!UseRegulator" );


## Train Methods

In [ ]:
factory->TrainAllMethods();

## Test and Evaluate Methods

In [ ]:
factory->TestAllMethods();

factory->EvaluateAllMethods();    

## Plot ROC Curve

In [ ]:
%jsroot
TCanvas *c1=factory->GetROCCurve(loader1);
c1->Draw();


In [ ]:
TCanvas *c2=factory->GetROCCurve(loader2);
c2->Draw();

## Cleanup

In [ ]:
std::cout << "==> TMVAClassification is done!" << std::endl;
outputFile->Close();
delete factory;
delete loader1;
delete loader2;